In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../helper_functions/")

import recency_util_live

import sys
sys.path.append("/home/ray__/ssd/BERT/") # gpt model utils location
from gpt_feat_utils import GPT_Inference


[nltk_data] Downloading package wordnet to /tmp/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /tmp/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /tmp/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [3]:
import pickle

## initialise artifacts

#gpt_model = GPT_Inference("/home/ray__/ssd/BERT/models/se/epoch3/", device="cpu") # gpt model location.
domain = "se"
artifacts_dir = "/home/ray__/ssd/minds/" + domain + "/recency/" # location which contains com_map, sent_dict, kp_entity_graph, gc, lc, entity and label_dict

# During call

In [9]:
def computing_groups(request, artifacts_dir):    
    group, group_ent_map_rank_lc, group_ent_map_rank_gc, gc, lc = recency_util_live.compute_groups_new_call(request, artifacts_dir, True)
    return group, group_ent_map_rank_lc, group_ent_map_rank_gc, gc, lc

In [5]:
def get_filtered_groups(group):
    filtered_group = {}
    count = 0
    for groupid, freq in sorted(group_ent_map_rank_lc.items(), key=lambda kv:kv[1], reverse=True):
        if count !=5:
            filtered_group[groupid] = group[groupid]
        else:
            pass

    for groupid, freq in sorted(group_ent_map_rank_gc.items(), key=lambda kv:kv[1], reverse=True):
        if count !=5:
            filtered_group[groupid] = group[groupid]
        else:
            pass
    return filtered_group
 

In [6]:
# for groupid, groupobj in filtered_group.items():
#     print ("\n\nGroupid: ", groupid, "\n\n")
#     seg_list = [seg["originalText"] for seg in groupobj]
#     print (*seg_list, sep="\n\n")

In [29]:
# lc = pickle.load(open("/home/ray__/ssd/minds/se/recency/lc.pkl","rb"))

# Post call - graph related updates

In [7]:
import sys
sys.path.append("/home/ray__/ssd/pos_tag/code/") # Custom pos tag location.
sys.path.append("/home/ray__/ssd/") # ner util code
import recency_util_post

from bert_ner_utils_graph import BERT_NER
from distilbert_pos_tagger import DistilBertPosTagger

pos_tagger = DistilBertPosTagger("/home/ray__/ssd/pos_tag/model/Distilbert/","cpu")
ner_model = BERT_NER('/home/ray__/ssd/bert-ner/',labels = ["O", "MISC", "MISC",  "PER", "PER", "ORG", "ORG", "LOC", "LOC"],device="cpu")

{"level": "info", "filename": "file_utils.py", "lineno": 38, "module": "file_utils", "ts": "2020-02-21T06:03:28.057381Z", "msg": "PyTorch version 1.1.0 available."}
{"level": "warning", "filename": "tokenization_openai.py", "lineno": 96, "module": "tokenization_openai", "ts": "2020-02-21T06:03:40.536544Z", "msg": "ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy."}
{"level": "info", "filename": "tokenization_utils.py", "lineno": 416, "module": "tokenization_utils", "ts": "2020-02-21T06:03:40.719943Z", "msg": "Adding _start_ to the vocabulary"}
{"level": "info", "filename": "tokenization_utils.py", "lineno": 416, "module": "tokenization_utils", "ts": "2020-02-21T06:03:40.720807Z", "msg": "Adding _delimiter_ to the vocabulary"}
{"level": "info", "filename": "tokenization_utils.py", "lineno": 416, "module": "tokenization_utils", "ts": "2020-02-21T06:03:40.721629Z", "msg": "Adding _classify_ to the vocabulary"}


In [11]:
def update_mind_artifacts(filtered_group, artifacts_dir):
    master_paragraphs, master_ids = recency_util_post.get_grouped_segments(filtered_group)
    multi_label_dict = pickle.load(open(artifacts_dir + "label_dict.pkl","rb"))
    ent_feat_dict = pickle.load(open(artifacts_dir + "entity.pkl","rb"))

    ent_sent_dict, kp_sent_dict, label_dict, noun_list, entity_dict = recency_util_post.form_sentence_graph(master_paragraphs, master_ids, multi_label_dict)
    ent_single_label_dict, all_sent_dict = recency_util_post.form_single_label_dict(label_dict, ent_sent_dict, kp_sent_dict)
    kp_entity_graph = recency_util_post.get_base_graph(artifacts_dir)
    kp_entity_graph, node_list = recency_util_post.update_entity_nodes(kp_entity_graph, ent_sent_dict, multi_label_dict)
    kp_entity_graph, node_list = recency_util_post.update_kp_nodes(kp_entity_graph, ent_sent_dict, node_list, kp_sent_dict)
    kp_entity_graph = recency_util_post.update_edges(kp_entity_graph, node_list, all_sent_dict, artifacts_dir)
    kp_entity_graph = recency_util_post.update_kp_tokens(kp_entity_graph, node_list)
    ent_kp_graph = recency_util_post.store_final_graph(kp_entity_graph, artifacts_dir)
    entity_dict = recency_util_post.update_entity_feat_dict(ent_sent_dict, ent_feat_dict)
    return 

In [50]:
# pickle.dump(kp_entity_graph, open("/home/ray__/ssd/minds/se/recency/kp_entity_graph.pkl","wb"))

# pipelined Testing

In [ ]:
# test it on a meeting

test_set = ['../topic_testing/validation_3.txt']

for file in test_set:
    with open(file,'rb') as f:
        request = json.load(f)
        if isinstance(request, str):
            request = json.loads(request)

    group, group_ent_map_rank_lc, group_ent_map_rank_gc, gc, lc = computing_groups(request, artifacts_dir)
    filtered_group = get_filtered_groups(group)
    update_mind_artifacts(filtered_group, artifacts_dir)

Using  01dadp74wfv607knpcb6vvxgtg  for feature extraction
{"level": "info", "filename": "scorer.py", "lineno": 122, "module": "scorer", "batches count": 1, "number of sentences": 280, "ts": "2020-02-21T06:18:33.874862Z", "msg": "computing in batches"}
{"level": "info", "filename": "scorer.py", "lineno": 125, "module": "scorer", "iteration count:": 0, "ts": "2020-02-21T06:18:33.878433Z", "msg": "getting feature vector"}
{"level": "info", "filename": "scorer.py", "lineno": 133, "module": "scorer", "iteration count": 0, "ts": "2020-02-21T06:18:49.481880Z", "msg": "Request Sent"}
('When data prototype back to users have them play around with it and then give you feedback here.', '2020-02-10T09:07:27Z', 'fb52cb663aec4795aee38ccfd904d315', '01eef9d1-15c1-4fc8-a27e-a04d8708cb5c') ('Does is understanding the user and understanding the market right as a p.m. Youre building a great product for your users.', '2020-02-10T09:03:28Z', 'fb52cb663aec4795aee38ccfd904d315', 'b082924b-f39e-4589-aafa-c17f

('Now we are going to do is multiply around and see if we can reduce this error.', '2020-02-10T09:32:27Z', 'fb52cb663aec4795aee38ccfd904d315', '213fdadd-9691-4ddd-b7ab-d4c69b56bc22') ('The higher we are the larger error is so descending means reducing the error.', '2020-02-10T09:34:17Z', 'fb52cb663aec4795aee38ccfd904d315', '6dc33819-022b-457b-85cc-323cde8e89d3')
('Now we are going to do is multiply around and see if we can reduce this error.', '2020-02-10T09:32:27Z', 'fb52cb663aec4795aee38ccfd904d315', '213fdadd-9691-4ddd-b7ab-d4c69b56bc22') ('This is equivalent to move line upwards and getting closer to the three points.', '2020-02-10T09:34:17Z', 'fb52cb663aec4795aee38ccfd904d315', '6dc33819-022b-457b-85cc-323cde8e89d3')
('Now we are going to do is multiply around and see if we can reduce this error.', '2020-02-10T09:32:27Z', 'fb52cb663aec4795aee38ccfd904d315', '213fdadd-9691-4ddd-b7ab-d4c69b56bc22') ('I think of linear regression is a plane come look at your data and draw the best fi

('We had the bucket size feature to the list used to train our model.', '2020-02-10T08:59:36Z', 'fb52cb663aec4795aee38ccfd904d315', '3d548bf8-b8cd-4f00-a55a-0b78b5f716ef') ('We use a data set from the US Census as an example.', '2020-02-10T08:57:42Z', 'fb52cb663aec4795aee38ccfd904d315', '8fbb3b8c-53e0-4c53-b360-8d2804cf45fc')
('We had the bucket size feature to the list used to train our model.', '2020-02-10T08:59:36Z', 'fb52cb663aec4795aee38ccfd904d315', '3d548bf8-b8cd-4f00-a55a-0b78b5f716ef') ('And the goal is to predict the income is greater than XnumberX,XnumberX based on attributes like their age and occupation.', '2020-02-10T08:57:42Z', 'fb52cb663aec4795aee38ccfd904d315', '8fbb3b8c-53e0-4c53-b360-8d2804cf45fc')
('We had the bucket size feature to the list used to train our model.', '2020-02-10T08:59:36Z', 'fb52cb663aec4795aee38ccfd904d315', '3d548bf8-b8cd-4f00-a55a-0b78b5f716ef') ('Youll also find complete code to train a tensorflow estimator on this data set.', '2020-02-10T08:57

('It can be broken down into Is there a market research business strategy and business modeling finding a great user experience by a product getting the product built from a technical point of view and then finally taking the product to market.', '2020-02-10T09:03:28Z', 'fb52cb663aec4795aee38ccfd904d315', 'b082924b-f39e-4589-aafa-c17fecf1771f') ('The next step is building a prototype which in layman is term is like a working model of your products.', '2020-02-10T09:05:27Z', 'fb52cb663aec4795aee38ccfd904d315', 'b56d1fb7-43c6-46c1-b520-c4ab0422cd5b')
('Does is understanding the user and understanding the market right as a p.m. Youre building a great product for your users.', '2020-02-10T09:03:28Z', 'fb52cb663aec4795aee38ccfd904d315', 'b082924b-f39e-4589-aafa-c17fecf1771f') ('You need to understand the market in and out and what YouTube editor offers as state abbreviations, especially be scenario the competitor could mean a big difference between winning the sale or losing the next Simone

{"level": "info", "filename": "grouper_segments.py", "lineno": 397, "module": "grouper_segments", "outlier threshold is : ": 0.4356013536453247, "ts": "2020-02-21T06:18:57.094563Z", "msg": "Outlier Score"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 1041, "module": "grouper_segments", "edges before prunning": 2521, "edges after prunning": 2521, "modularity": 0.7273036890327346, "ts": "2020-02-21T06:18:59.283252Z", "msg": "Meeting Graph results"}
cluster before alteration=========>
When data prototype back to users have them play around with it and then give you feedback here.
The reviews is take the feed bags and kind of to the process again, unless you till you feel that you have kind of hit the nail on the head and I am ready to move on to the next stage of actually participate.
So now comes the build phase which is the grunt work with.
It can be broken down into Is there a market research business strategy and business modeling finding a great user experience by a

See this is about your empowerment make decisions and change directions versus Going to go through all the sort of a lengthy Change Control process or get a ton of sign off. d2625f65-97a4-47de-be2a-23c28bafcc69 

Were smart people we read but what we do often lack that we cannot get out of a document is product Vision sort of that the themes principles and guidelines for why we are even directly creating this product. d2625f65-97a4-47de-be2a-23c28bafcc69 

We want to know that you do not have to always go and ask the knowledgeable person when we answer to a question and we also because telephone communication we also want d2625f65-97a4-47de-be2a-23c28bafcc69 

We are looking for a single point of contact and we do not want to have to run all of it organization trying to figure out get a question answered or something like that. d2625f65-97a4-47de-be2a-23c28bafcc69 

We need an enthusiastic figurehead for the project person that really encapsulates embodies. d2625f65-97a4-47de-be2a-23c2

Relevant sentence:  They are remembering the things where we went a little bit above and beyond to make the product better to sort of show you really care and it is powerful idea.    =====    I do not remember whether I was car like train I mean, so it is pretty right that was pretty enough to take the picture and serving its purpose in some sense, but The memorability is much lacking right now build a flat product which would take the picture that is good, but not particularly notable.
order difference: 0
Relevant sentence:  I do not remember whether I was car like train I mean, so it is pretty right that was pretty enough to take the picture and serving its purpose in some sense, but The memorability is much lacking right now build a flat product which would take the picture that is good, but not particularly notable.    =====    Anyway, you are not necessarily doing it not you are not doing what the product is.
order difference: 0
Relevant sentence:  Anyway, you are not necessarily 

order difference: 0
Relevant sentence:  See this is about your empowerment make decisions and change directions versus Going to go through all the sort of a lengthy Change Control process or get a ton of sign off.    =====    Were smart people we read but what we do often lack that we cannot get out of a document is product Vision sort of that the themes principles and guidelines for why we are even directly creating this product.
order difference: 0
Relevant sentence:  Were smart people we read but what we do often lack that we cannot get out of a document is product Vision sort of that the themes principles and guidelines for why we are even directly creating this product.    =====    We want to know that you do not have to always go and ask the knowledgeable person when we answer to a question and we also because telephone communication we also want
order difference: 0
Relevant sentence:  We want to know that you do not have to always go and ask the knowledgeable person when we answ

[(0, 0), (7, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (12, 5), (6, 6), (8, 7), (9, 8), (10, 9), (11, 10)]
[[["Look at a simple example this three points. We're going to try to find the best line that fits through those three points. Obviously Best Line subjective, but we try to find a line that works. Well, since we're teaching the computer how to do it computer company, eyeball the line so as to get it to draw a random line, And then see how fast this thing is. So in order to see how bad the line is calculate the error. So we're going to protecting the error look at the length of the distances from the line to the three points and respond to Simply say the error of this line is the sum of those three read lengths. Now we're going to do is multiply around and see if we can reduce this error. So let's say we move in this direction. And we calculate the error given by the yellow distances. We add them up and realize that will increase the error. So that's not a good direction to go. L

{'1': [(48, 25), (49, 16), (1, 7), (7, 6), (471, 4), (11, 3), (35, 2), (28, 2)], '6': [(11, 45), (28, 12), (30, 3), (57, 3), (49, 3)], '4': [(21, 10), (48, 8), (57, 6), (11, 5), (30, 4), (1, 3), (17, 2)], '3': [(28, 13), (11, 10), (21, 7), (48, 6), (57, 5), (50, 4), (7, 3), (434, 2)], '5': [(11, 36), (57, 17), (28, 4), (21, 2)], '9': [(48, 27), (57, 9), (11, 7), (1, 6)], '2': [(7, 6), (33, 6), (3, 3), (57, 2)], '10': [(11, 26), (7, 14), (28, 13), (57, 7), (21, 4), (941, 2)], '0': [(49, 17), (7, 6), (57, 6), (28, 4), (48, 3), (357, 2)], '12': [(11, 16), (28, 16), (57, 12), (21, 11), (33, 2)], '7': [(21, 6), (7, 4), (11, 2), (28, 2), (49, 2)], '11': [(21, 8), (58, 8), (57, 7), (48, 6), (33, 3), (7, 3), (50, 3), (1, 3), (28, 2)], '8': [(30, 11), (21, 11), (33, 4), (28, 4), (11, 2), (14, 2)]}
writing the gc and lc update.


In [ ]:
for groupid, groupobj in filtered_group.items():
    print ("\n\nGroupid: ", groupid, "\n\n")
    seg_list = [seg["originalText"] for seg in groupobj]
    print (*seg_list, sep="\n\n")